In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the files
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
print(type(train_df))

In [ ]:
from fastai.vision.all import *

In [ ]:
import torch.nn as nn

In [ ]:
simpleNN = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256,10)
)

In [ ]:
import torch.optim as optim

In [ ]:
# standard loss function for multi-class classification
criterion = nn.CrossEntropyLoss()

# Use SGD as optimizer
optim = optim.SGD(simpleNN.parameters(), lr = 0.001)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
# create custom Pytorch Dataset
class CustomMNISTDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        #transform needs to be defined
        self.transform = transform
        
    def __len__(self):
        #return number of samples
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # first column is the label
        label = torch.tensor(int(row[0]))
        image = torch.tensor(row[1:].values, dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [ ]:
# create custom Pytorch Dataset for test set (no labels)
class CustomMNISTTestDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        #transform needs to be defined
        self.transform = transform
        
    def __len__(self):
        #return number of samples
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image = torch.tensor(row.values, dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
            
        return image # Return the index as image ID

In [ ]:
# customize scaling for flattened tensor
class NormalizeFlattenedTensor:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        # Normalize the tensor
        tensor = (tensor - self.mean) / self.std
        return tensor

In [ ]:
import torchvision.transforms as transforms

# Custom scaling transform
transform = NormalizeFlattenedTensor(mean=127.5, std=127.5)

pro_train = CustomMNISTDataset(train_df, transform=transform)
pro_test = CustomMNISTTestDataset(test_df, transform=transform)

train_loader = DataLoader(pro_train, batch_size=64, shuffle=True)
test_loader = DataLoader(pro_test, batch_size=64, shuffle=False)

In [ ]:
num_epochs = 50

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    simpleNN.train() # set to training mode
    for images, labels in train_loader:
        
        # Reset gradients to 0
        optim.zero_grad()
        
        #Forward pass
        predictions = simpleNN(images)
        loss = criterion(predictions, labels)
        
        #Backward pass
        loss.backward()
        
        #Update weights
        optim.step()
        
        #Accumulate loss
        running_loss += loss.item()
    
    #Print average loss per epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

In [ ]:
# Model Evaluation - Test against test set
simpleNN.eval() # set model to evaluation mode

predictions = [] # store predictions
global_idx = 1

with torch.no_grad(): # disable gradient calculation
    for images in test_loader:
        outputs = simpleNN(images)
        _, predicted = torch.max(outputs, 1) # get the indices of the max logit score (confidence)
        batch_size = images.size(0)
        for i in range(batch_size):
            predictions.append((global_idx, predicted[i].item()))
            global_idx += 1

In [ ]:
submission_df = pd.DataFrame(predictions, columns=['ImageId', 'Label'])
submission_df.to_csv('submission.csv', index=False)